# AgentAI Introduction

This library provides a simple, familiar and flexible way to combine Python functions with OpenAI functions. AgentAI builds on top of the OpenAI API, offering utilities to make it easier to manage conversations and implement function calls guided by AI responses.

## Introduction

The `agentai` library enables you to maintain the state of a conversation, invoke the model for generating responses, and parse the model's responses to generate function arguments that adhere to a given function specification. It also allows you to manage function specifications and execute functions whose inputs are model-generated. In this guide, we'll walk you through the library's various functionalities and demonstrate how to utilize them effectively.

## What to expect

We'll start by introducing the basic concepts: 
1. Tool
1. Conversation
2. An Function Specification

Then, we will create a real-world application where we integrate model-generated arguments into function executions, demonstrating the interaction between the model and a SQLite database.

## What is a Tool Decorator? <a name="tool-decorator"></a>

A tool decorator from the `agentai` library is a Python decorator. OpenAI Functions are specified with the following fields:

- **Name:** The name of the function.
- **Description:** A description of what the function does. The model will use this to decide when to call the function.
- **Parameters:** The parameters object contains all of the input fields the function requires. These inputs can be of the following types: String, Number, Boolean, Object, Null, AnyOf. Refer to the [API reference docs](https://platform.openai.com/docs/api-reference/chat) for details.
- **Required:** Which of the parameters are required to make a query. The rest will be treated as optional.

These are auto-generated from a Python function when annotated with a `@tool`. 

In [1]:
from IPython.display import display, Markdown, Latex
import json

%load_ext autoreload
%autoreload 2

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

## Utilities

First let's import a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [2]:
from agentai.api import chat_complete
from agentai.conversation import Conversation
from agentai.function_parser import tool

## Basic concepts

Next we'll create a specification for a function called ```get_current_weather```. Later we'll pass this function specification to the API in order to generate function arguments that adhere to the specification.

In [3]:
from enum import Enum


class TemperatureUnit(Enum):
    celsius = "celsius"
    fahrenheit = "fahrenheit"


@tool
def get_current_weather(location: str, format: TemperatureUnit) -> str:
    """
    Get the current weather

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        format (str): The temperature unit to use. Infer this from the users location.

    Returns:
        str: The current weather
    """
    # Your function implementation goes here.
    return ""


registered_functions = [get_current_weather]
functions = [json.loads(func.json_info) for func in registered_functions]

In [4]:
conversation = Conversation()
conversation.add_message("user", "what is the weather like today?")

message = {"role": "assistant", "content": None}
chat_response = chat_complete(conversation.conversation_history, functions=functions, model=GPT_MODEL)
message = chat_response.json()["choices"][0]["message"]
while message["content"] is None:
    print("No response from assistant, trying again")
    chat_response = chat_complete(conversation.conversation_history, functions=functions, model=GPT_MODEL)
    message = chat_response.json()["choices"][0]["message"]
conversation.add_message(message["role"], message["content"])
message

{'role': 'assistant', 'content': 'Where are you currently located?'}

In [5]:
conversation.conversation_history

[{'role': 'user', 'content': 'what is the weather like today?'},
 {'role': 'assistant', 'content': 'Where are you currently located?'}]

In [6]:
# Once the user provides the required information, the model can generate the function arguments
conversation.add_message("user", "I'm in Bengaluru, India")
chat_response = chat_complete(conversation.conversation_history, functions=functions, model=GPT_MODEL)

eval(chat_response.json()["choices"][0]["message"]["function_call"]["arguments"])
# chat_response.json()

{'location': 'Bengaluru, India', 'format': 'celsius'}

## Integrating API calls with function execution

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation use cases are high-risk in a production environment - models can be unreliable when generating consistent SQL syntax. A more reliable way to solve this problem may be to build a query generation API that takes the desired columns as input from the model.

### Pull SQL Database Info

First let's define some helpful utility functions to extract data from a SQLite database.

In [7]:
import sqlite3

conn = sqlite3.connect("../data/Chinook.db")
print("Opened database successfully")

Opened database successfully


Now can use these utility functions to extract a representation of the database schema.

In [8]:
from agentai.sqlite_utils import DBUtils

database_schema_dict = DBUtils(conn).get_database_info()
database_schema_string = "\n".join(
    [f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}" for table in database_schema_dict]
)

In [9]:
print(database_schema_string)

Table: Album
Columns: AlbumId, Title, ArtistId
Table: Artist
Columns: ArtistId, Name
Table: Customer
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: Employee
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: Genre
Columns: GenreId, Name
Table: Invoice
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: InvoiceLine
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: MediaType
Columns: MediaTypeId, Name
Table: Playlist
Columns: PlaylistId, Name
Table: PlaylistTrack
Columns: PlaylistId, TrackId
Table: Track
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice


As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [10]:
@tool
def ask_database(query: str) -> str:
    """
    Use this function to answer user questions about music. Input should be a fully formed SQL query.

    Args:
        query (str):SQL query extracting info to answer the user's question.
                    SQL should be written using this database schema:
                    Table: Album
                    Columns: AlbumId, Title, ArtistId
                    Table: Artist
                    Columns: ArtistId, Name
                    Table: Customer
                    Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
                    Table: Employee
                    Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
                    Table: Genre
                    Columns: GenreId, Name
                    Table: Invoice
                    Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
                    Table: InvoiceLine
                    Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
                    Table: MediaType
                    Columns: MediaTypeId, Name
                    Table: Playlist
                    Columns: PlaylistId, Name
                    Table: PlaylistTrack
                    Columns: PlaylistId, TrackId
                    Table: Track
                    Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice

                    IMPORTANT: Please return a fixed SQL in PLAIN TEXT.
                    Your response should consist of ONLY the SQL query.

    Returns:
        str: _description_
    """
    try:
        results = conn.execute(query).fetchall()
        return results
    except Exception as e:
        raise Exception(f"SQL error: {e}")


agentai_functions = [json.loads(func.json_info) for func in [ask_database]]

### SQL execution

Now let's implement the function that the agent will use to query the database. We also need to implement utilities to integrate the calls to the Chat Completions API with the function it is calling.

In [11]:
from agentai.api import chat_complete_execute_fn

In [12]:
agent_system_message = """You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!"""

sql_conversation = Conversation()
sql_conversation.add_message(role="system", content=agent_system_message)
sql_conversation.add_message("user", "Hi, who are the top 5 artists by number of tracks")
assistant_message = chat_complete_execute_fn(
    conversation=sql_conversation, functions=agentai_functions, model=GPT_MODEL, callable_function=ask_database
)

In [13]:
sql_conversation.display_conversation(detailed=True)

system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: The top 5 artists by the number of tracks are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks




In [14]:
sql_conversation.add_message("user", "What is the name of the album with the most tracks")
chat_response = chat_complete_execute_fn(
    conversation=sql_conversation, functions=agentai_functions, model=GPT_MODEL, callable_function=ask_database
)

In [15]:
sql_conversation.display_conversation(detailed=True)

system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: The top 5 artists by the number of tracks are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks


user: What is the name of the album with the most tracks


function: [('Greatest Hits', 57)]


assistant: The album with the most tracks is "Greatest Hits" with 57 tracks.


